# DB to Python
db에 있는 iris 테이블을 data로 읽고 정제 후, 분류까지 진행한다.  
과제는 이 과정을 db_score 데이터로 반복하는 것  
- Updated 1009

mysql에서 새로 만든 db 스키마 practice에 iris table 을 생성해서 데이터를 넣어본다.

In [23]:
# ready
import pymysql
import sqlalchemy
import pandas as pd
import numpy as np

# sklearn
from sklearn.model_selection import train_test_split,KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

### database 연결 후 iris table 불러들이기

In [2]:
# database
conn = pymysql.connect(host='localhost', user='root', password='1234', db='practice')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql = '''
    select * from iris
    '''
curs.execute(sql)
# data
data = curs.fetchall()
print(data)

# close
curs.close()
conn.close()

[{'index': 0, 'sepal_length': 5.1, 'sepal_width': 3.5, 'petal_length': 1.4, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 1, 'sepal_length': 4.9, 'sepal_width': 3.0, 'petal_length': 1.4, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 2, 'sepal_length': 4.7, 'sepal_width': 3.2, 'petal_length': 1.3, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 3, 'sepal_length': 4.6, 'sepal_width': 3.1, 'petal_length': 1.5, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 4, 'sepal_length': 5.0, 'sepal_width': 3.6, 'petal_length': 1.4, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 5, 'sepal_length': 5.4, 'sepal_width': 3.9, 'petal_length': 1.7, 'petal_width': 0.4, 'species': 'setosa'}, {'index': 6, 'sepal_length': 4.6, 'sepal_width': 3.4, 'petal_length': 1.4, 'petal_width': 0.3, 'species': 'setosa'}, {'index': 7, 'sepal_length': 5.0, 'sepal_width': 3.4, 'petal_length': 1.5, 'petal_width': 0.2, 'species': 'setosa'}, {'index': 8, 'sepal_length': 4.4, 'sepal_width': 2.9, 'petal_le

### Data Generation

In [3]:
# 데이터 모양 확인
data[:2]

[{'index': 0,
  'sepal_length': 5.1,
  'sepal_width': 3.5,
  'petal_length': 1.4,
  'petal_width': 0.2,
  'species': 'setosa'},
 {'index': 1,
  'sepal_length': 4.9,
  'sepal_width': 3.0,
  'petal_length': 1.4,
  'petal_width': 0.2,
  'species': 'setosa'}]

In [4]:
# list comprehension 이용
X = [(t['sepal_length'],t['sepal_width'],t['petal_length'],t['petal_width']) for t in data]
X = np.array(X)

y = [1 if (t['species'] == 'versicolor') else -1 for t in data]
y = np.array(y)

# 확인
X,y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [16]:
# split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_train.shape, X_test.shape

((90, 4), (60, 4))

In [17]:
# model fit
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

# predict
y_pred = clf.predict(X_test)

In [18]:
# evaluate : accuracy_score,precision_score,recall_score,f1_score
acc = accuracy_score(y_test,y_pred)
pr = precision_score(y_test,y_pred)
rc = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)

print(f'Accracy : {acc}')
print(f'Precision : {pr}')
print(f'Recall : {rc}')
print(f'F1 : {f1}')

Accracy : 0.9666666666666667
Precision : 0.9473684210526315
Recall : 0.9473684210526315
F1 : 0.9473684210526315


iris data 는 너무 좋기 때문에 test size 를 0.3 이상으로 주면 모든 평가지표가 1이 나오도록 잘 맞추게 된다   
비교하기 위해서 일단 test_size=0.4로 설정함

### Performance Function

In [25]:
def clf_eval(y_true,y_pred):
    
    # True/False & Posi&Neg
    tp,tn,fp,fn = 0,0,0,0
    
    for y,yp in zip(y_true,y_pred):
        if y == 1 and yp == 1:
            tp += 1
        elif y == 1 and yp == -1:
            fn += 1
        elif y == -1 and yp == 1:
            fp += 1
        else:
            tn += 1
    
    # accuracy
    acc = (tp+tn) / (tp+tn+fp+fn)
    pre = (tp) / (tp+fp)       # positive 로 예측한 것 중
    rec = (tp) / (tp+fn)      # 실제 positive 중 예측한 것
    f1 = (2*pre*rec) / (pre+rec)
    
    print(f'Accracy : {acc}')
    print(f'Precision : {pre}')
    print(f'Recall : {rec}')
    print(f'F1 : {f1}')
    return acc,pre,rec,f1

In [26]:
acc,pre,rec,f1 = clf_eval(y_test,y_pred)

Accracy : 0.9666666666666667
Precision : 0.9473684210526315
Recall : 0.9473684210526315
F1 : 0.9473684210526315


**metrics 와 비교 -> 동일함**  
Accracy : 0.9666666666666667  
Precision : 0.9473684210526315  
Recall : 0.9473684210526315  
F1 : 0.9473684210526315  

### K-Fold

In [31]:
# setting
cv=5     # 10

kf = KFold(n_splits=cv, random_state=42, shuffle=True)
accs = []
pres = []
recs = []
f1s = []

# roop
for t,v in kf.split(X):
    X_train,X_val = X[t],X[v]
    y_train,y_val = y[t],y[v]
    
    # model 
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_val)
    
    # eval
    clf_eval(y_val,y_pred)
    print(''' ''')
    # save
    accs.append(acc)
    pres.append(pre)
    recs.append(rec)
    f1s.append(f1)

# mean
np.mean(accs),np.mean(pres),np.mean(recs),np.mean(f1s)

Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9666666666666667
Precision : 0.9090909090909091
Recall : 1.0
F1 : 0.9523809523809523
 
Accracy : 0.9333333333333333
Precision : 1.0
Recall : 0.8
F1 : 0.888888888888889
 
Accracy : 0.9333333333333333
Precision : 0.9
Recall : 0.9
F1 : 0.9
 
Accracy : 0.9333333333333333
Precision : 0.8461538461538461
Recall : 1.0
F1 : 0.9166666666666666
 


(0.9666666666666666,
 0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315)